### Set TACC account credentials for authenticating to the Tapis TACC Tenant 

In [ ]:
import getpass
permitted_username = 'wongy'
permitted_user_password = 'Jw@101270'
# permitted_username = getpass.getpass(prompt='Username: ', stream=None)
# permitted_user_password = getpass.getpass(prompt='Password: ', stream=None)

#Set Tapis Tenant and Base URL
tenant="tacc"
base_url = 'https://' + tenant + '.tapis.io'

#Load Python SDK
from tapipy.tapis import Tapis

#Create python Tapis client for user
permitted_client = Tapis(base_url= base_url, 
                         username=permitted_username,
                         password=permitted_user_password, 
                         account_type='user', 
                         tenant_id=tenant
                        ) 


#Generate an Access Token that will be used for all API calls
permitted_client.get_tokens()

permitted_client.access_token

### Listing All Existing Projects
- To check if streams api is working correctly

In [ ]:
permitted_client.streams.list_projects()

### Setup Streams Variables that are used in the rest of the notebook

In [ ]:
import datetime
storage_id = "dev.tapisv3.storage.system"
# project_id = 'Mesonet' + str(datetime.datetime.today().isoformat())
project_id = 'Mesonet2022-11-22T09:02:01.224241'
site_id = 'KulaAg'
instrument_id = '0119_KulaAg'

### Creating the MesoNet Project

In [ ]:
result = permitted_client.streams.create_project(project_name=project_id,
                                                        description='TEST project for MesoNet',
                                                        owner='testuser2', pi='wongy', 
                                                        funding_resource='test', 
                                                        project_url='https://www.hawaii.edu/climate-data-portal/',
                                                        active=True,_tapis_debug=True)

result


### Creating the Sites in Bulk

In [ ]:
## Create a Streams Site
result, debug = permitted_client.streams.create_site(project_id=project_id,
                                                     request_body=[{
                                                     "site_name":site_id, 
                                                     "site_id":site_id,
                                                     "latitude":50, 
                                                     "longitude":10, 
                                                     "elevation":2,
                                                     "description":'test_site'
                                                    }], _tapis_debug=True)
print(result)

In [ ]:
result = permitted_client.streams.list_sites(project_id=project_id)

result

### Create Instruments

In [ ]:
result, debug = permitted_client.streams.create_instrument(project_id=project_id,
                                                           site_id=site_id,
                                                           request_body=[{
                                                            "inst_name": instrument_id,
                                                            "inst_id": instrument_id,
                                                            "inst_description": "test instrument for MesoNet Project"
                                                           }],
                                                           _tapis_debug=True)

result

In [12]:
result = permitted_client.streams.list_instruments(project_id=project_id, site_id=site_id)

result

[
 chords_id: 92
 created_at: 2022-11-22 22:43:59.546906
 inst_description: test instrument for MesoNet Project
 inst_id: 0119_KulaAg
 inst_name: 0119_KulaAg
 last_updated: 2022-11-22 22:43:59.546933]

### Parsing Variable Names from ikewai MesoNet file

In [15]:
import urllib

link = 'https://ikeauth.its.hawaii.edu/files/v2/download/public/system/ikewai-annotated-data/HCDP/raw/2022/10/25/0151_ParkHQ_MinMax.dat'

data_file = urllib.request.urlopen(link).readlines()

list_vars = data_file[1].decode("UTF-8").strip().replace("\"", "").split(",")

list_vars

['TIMESTAMP',
 'RECORD',
 'Tair_1_Min',
 'Tair_1_Max',
 'Tair_2_Min',
 'Tair_2_Max',
 'TC_air_Min',
 'TC_air_Max',
 'RH_1_Min',
 'RH_1_Max',
 'RH_2_Min',
 'RH_2_Max',
 'WindGust_Max',
 'WindGust_TMx']

### Creating the Variables in Tapis

In [ ]:
result, debug = permitted_client.streams.create_variable(project_id=project_id, site_id=site_id, inst_id=instrument_id)

### Parsing Measurements from MesoNet file

In [18]:
# for i in range(3, len(data_file)):
#     data_file[i].decode("UTF-8").strip().split(",")
#     pass

variables = []

measurements = {}

row = 4

# for row in range(4, len(data_file)):

for i in range(0, len(list_vars)):
    if(list_vars[i] == 'TIMESTAMP'):
        measurements["datetime"] = data_file[row].decode("UTF-8").strip().split(",")[i]
    else:
        measurements[list_vars[i]] = data_file[row].decode("UTF-8").strip().split(",")[i]

variables.append(measurements)

measurements

{'datetime': '"2022-10-25 00:05:00"',
 'RECORD': '49658',
 'Tair_1_Min': '11.92676',
 'Tair_1_Max': '12.26837',
 'Tair_2_Min': '11.94061',
 'Tair_2_Max': '12.28222',
 'TC_air_Min': '11.96775',
 'TC_air_Max': '12.26832',
 'RH_1_Min': '78.86103',
 'RH_1_Max': '80.63091',
 'RH_2_Min': '81.19104',
 'RH_2_Max': '83.30367',
 'WindGust_Max': '8.493333',
 'WindGust_TMx': '"2022-10-25 00:03:53"'}

In [19]:
permitted_client.streams.create_measurement(inst_id=instrument_id, vars=variables)

ServerDownError: message: Unrecognized exception type: <class 'KeyError'>. Exception: 'variables'

In [22]:
permitted_client.streams.list_variables(project_id=project_id, site_id=site_id, inst_id=instrument_id)

[]

### A way to Delete all the projects

In [ ]:
# list_proj = permitted_client.streams.list_projects()

# for project in list_proj:
#     delete_id = getattr(project, 'project_id')
#     permitted_client.streams.delete_project(project_id=delete_id)